In [7]:
!pip install sentence_transformers

In [8]:
import pandas as pd
import torch
from sentence_transformers import SentenceTransformer


In [9]:
from google.colab import drive
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [10]:
filenames = ["CommentsMay2017.csv"]

In [11]:
def read_comments_from_dataset(filename):
  return pd.read_csv(filename)[["commentID", "commentBody", "commentSequence", "articleID", "parentID"]]

In [12]:
comment_sets = [(filename, read_comments_from_dataset("gdrive/MyDrive/Research/NewYorkTimesComments/" + filename)) for filename in filenames]


/usr/local/lib/python3.7/dist-packages/IPython/core/interactiveshell.py:2882: DtypeWarning: Columns (25,26) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


In [13]:
len(comment_sets)

1

In [14]:
model = SentenceTransformer('paraphrase-mpnet-base-v2')

In [15]:
from tqdm import tqdm

def compute_embeddings(comments):
    embeddings = []

    for comment in tqdm(comments.itertuples()):
      try:
        embedding = model.encode(comment.commentBody, convert_to_tensor=True)
      except:
        embedding = model.encode("None", convert_to_tensor=True)
      embeddings.append({"ID": comment.commentID,
                      "article_id": comment.articleID,
                      "commentBody": comment.commentBody,
                      "embedding": embedding})
    return embeddings

In [16]:
for comment_set in comment_sets:
  embeddings =  []
  embedding = compute_embeddings(comment_set[1])
  embeddings.extend(embedding)
  torch.save(embeddings, "gdrive/MyDrive/Research/embedded_comments_" + comment_set[0].split(".")[0] + "_paraphrase-mpnet-base-v2.pt")

276389it [47:34, 96.81it/s]
